In [1]:
from knowknow import *

In [2]:
import knowknow

In [3]:
from pymongo import MongoClient

In [4]:
db = MongoClient()['jstor-soc']

In [5]:
from load_db import db as dta

loading variable sociology-wos-74b/groups from disk
loading variable sociology-wos-74b/group_reps from disk
loading variable sociology-wos-74b/_attributes from disk


In [6]:
from pybtex.database.input import bibtex

#open a bibtex file
parser = bibtex.Parser()
bibdata = parser.parse_file("C:/Users/amcga/Downloads/my_deaths_fixed.bib")

In [7]:
rows = []
qs = []
#loop through the individual references
for bib_id in bibdata.entries:
    b = bibdata.entries[bib_id].fields
    try:
        # change these lines to create a SQL insert
        auths = ", ".join( str(x) for x in bibdata.entries[bib_id].persons["author"] )
        rows.append([
            auths,
            b["title"],
            b["year"]
        ])
        
        auths = [ x.last_names[-1] for x in bibdata.entries[bib_id].persons["author"] ]
        #print(auths)
        parts = auths
        
        q = ".*".join( parts )
        
        if bibdata.entries[bib_id].type != 'book' or True:
            #parts.append(b['year'])
            q += f" \({b['year']}\)"
            
        qs.append(q)
    # field may not exist for a reference
    except(KeyError):
        continue

In [8]:
[x for x in qs if 'Dohren' in x]

['Dohrenwend.*Dohrenwend \\(1969\\)']

In [9]:
qs[:5]

['Ajzen.*Fishbein \\(1980\\)',
 'Alwin.*Hauser \\(1975\\)',
 'Andrews \\(1967\\)',
 'Aronowitz \\(1973\\)',
 'Barber \\(1957\\)']

In [10]:
from tabulate import tabulate
#display(HTML(tabulate(rows, tablefmt='html')))

In [10]:
def format_cit(c):
    d = db['docs'].find_one( {'_id':c['doc']} )
    return "+ (%s, %s) %s" % (
        d['year'],
        d['journal'],
        "///".join( c['contextPure'].split("\n") )
    )

In [12]:
qstr = "Mead.*1928"
qstr = "Nettler.*1974"
qstr = "Nie et al.*"
qstr = "Bogue.*"
qstr = "Edwards.*197"
qstr = ".*Kolb.*"
qstr = "Merton.*Lazarsfeld.*\)"
qstr = "Andrews.*"
qstr = "Dohrenwend.*"
qstr = "Kardiner\s\("
qstr = "Loomis and Beegle .*"
qstr = "MacIver.*1937\)"
qstr = "Chapin.*\)"

q = {"citations":{"$elemMatch":{"$regex":qstr}}}
cits = list(db['cits'].find(q))

In [24]:
cit_reps = Counter([
    y
    for x in cits
    for y in x['citations']
    if re.match(qstr, y)
])

In [25]:
cit_reps.most_common(20)

[('Chapin (1974)', 11),
 ('Chapin (1997)', 7),
 ('Chapin (2004)', 6),
 ('Chapin (1955)', 5),
 ('Chapin (1934)', 3),
 ('Chapin (1911)', 3),
 ('Chapin (1938)', 2),
 ('Chapin (1933)', 2),
 ('Chapin (1883)', 2),
 ('Chapin (1935)', 2),
 ('Chapin (1947)', 1),
 ('Chapin (1928)', 1),
 ('Chapin et al. (2000)', 1),
 ('Chapin and Tsouderos (1955)', 1),
 ('Chapin (1910)', 1),
 ('Chapin and Tsouderos (1956)', 1),
 ('Chapin (1968)', 1),
 ('Chapin (2007)', 1),
 ('Chapin et al. (2004)', 1),
 ('Chapin (1924)', 1)]

In [32]:
def find_quote(s, with_who=None):
    q = {"contextPure":{"$regex":s}}
    if with_who is not None:
        q['citations'] = {"$elemMatch":{"$regex":with_who}}
        
    cit = db['cits'].find_one(q)
    doc = db['docs'].find_one({"_id":cit['doc']})
    print(f"{' '.join(doc['authors'])} {doc['title']} ({doc['year']}). {doc['journal']}, doc['fpage'], doc['lpage']")
    print(f"{'; '.join(cit['citations']).replace(')','').replace('(','')}")
    print(f"{cit['contextLeft']} <CITATION> {cit['contextRight']}")

In [14]:
find_quote("been called by others the \"cotton counties\" or \"plantation counties\"")

William W. Falk Bruce H. Rankin The Cost of Being Black in the Black Belt (1992). Social Problems, doc['fpage'], doc['lpage']
Mann 1990; Odum 1936; Vance 1932
see). As we show in Maps 1 and 2, these developments led to the settlement pattern still evident today. The "physical environment" and "spatial structure" of the Black Belt are rooted deeply in the agricultural heritage of the region, particularly in cotton production (see Hilliard 1979). What we call the "Black Belt" has often been called by others the "cotton counties" or "plantation counties" ( <CITATION> ). In addition to their cotton and plantation heritages, the counties are unique in that they have h


In [68]:
find_quote("conceptual precursors of the movement")

Abbott L. Ferriss The Quality of Life Concept in Sociology (2004). The American Sociologist, doc['fpage'], doc['lpage']
Ogburn 1933
ndicators, DHEW, 1969) While "welfare" was an initial motive, the term, quality of life (QOL) did not enter until later. The social indicator "movement" involves not only statistical time series but also national social reports that analyzed trends and interconnections of social forces. In the United States two conceptual precursors of the movement were Recent So cial Trends in the United States ( <CITATION> ) and Southern Regions of the United States (Odum, 1936), two sociologists. Two chiefly economic soc


In [69]:
find_quote("Neo-Marxian authors")

Seymour Spilerman Wealth and Stratification Processes (2000). Annual Review of Sociology, doc['fpage'], doc['lpage']
Wright & Perrone 1977; Wright 1985; Poulantzas 1975; Parkin 1978; Parkin 1978
e social order (Davis & Moore 1945; Parkin 1978, p. 610). Also, in Marxian theory, it is one's position in the productive system that underlies life chances and living standards. Neo-Marxian authors have expanded the traditional Marxian formulation by considering the unique situations of different occupational groups, yet location in the productive order remains the essential explanatory dynamic ( <CITATION> ). There is also an empirical basis to the neglect of wealth in stratification research since occupa


In [70]:
find_quote("Neo-Marxist contributions to new structuralism")

Michael Wallace Kevin T. Leicht Don Sherman Grant, II Positional Power, Class, and Individual Earnings Inequality: Advancing New Structuralist Explanations (1993). The Sociological Quarterly, doc['fpage'], doc['lpage']
Wright 1976; Wright 1979; Wright 1985; Wright and Perrone 1977; Robinson and Kelley 1979
nce, and is also positively related to earnings (Kalleberg, Wallace, and Althauser 1981). Neo-Marxist contributions to new structuralism center on the role of the class structure in capitalist society, specifically the antagonistic relations between, and changing fortunes

Positional Power, Class, and Individual Earnings Inequality 87 of, capitalists, petty bourgeoisie, managers, and proletariat ( <CITATION> ). Past research has consistently shown that class is conceptually and empirically distinct from occ


In [71]:
find_quote("functionalist stratification theory")

Max Haller Marriage, Women, and Social Stratification: A Theoretical Critique (1981). American Journal of Sociology, doc['fpage'], doc['lpage']
Parsons 1953; Parsons 1940; Parsons 1953; Barber 1957
ly the same social status from their wedding day onward. Only to the degree that they act together on common tasks or stick together against outsiders "to maintain proper fronts" (McCall and Simmons 1966, p. 176) can they be regarded as a unit.11 From this point of view, the model of the nuclear family and its place in the stratification system as developed by functionalist stratification theory ( <CITATION> ) seems highly questionable. Three assumptions of this model are relevant here: (1) individuals deri


In [16]:
find_quote("Fishbein and Ajzen's theory of reasoned action")

Thomas R. Prohaska Gary Albrecht Judith A. Levy Noreen Sugrue Joung-Hwa Kim Determinants of Self-Perceived Risk for AIDS (1990). Journal of Health and Social Behavior, doc['fpage'], doc['lpage']
Ajzen and Fishbein 1980
 Perception of risk is probably the most salient psychodynamic feature in the leading models of health 384

DETERMINANTS OF SELF-PERCEIVED RISK FOR AIDS 385 behavior. It is a fundamental component of the health belief model (Becker and Janz 1987; Janz and Becker 1984), the common sense model of illness danger (Leventhal, Meyer, and Nerenz 1980), and Fishbein and Ajzen's theory of reasoned action ( <CITATION> ), as well as a predisposing factor in the PRECEDE model (Green, Kreuter, Deeds, and Partridge 1979)


In [18]:
find_quote("Originally, empirical studies in this field were often guided by the assumption")

Jörg Rössel Conditions for the Explanatory Power of Life Styles (2008). European Sociological Review, doc['fpage'], doc['lpage']
Ajzen and Fishbein 1980
es on the one hand and social action on the other we enter a classical field of social psychological research. Here, research on the connection between attitudes and behaviour has a long tradition (Ajzen and Fishbein, 1980, 12-27). Originally, empirical studies in this field were often guided by the assumption that attitudes towards objects also come along with patterns of behaviour towards them ( <CITATION> ). However, numerous empirical tests have made clear that the statistical correlation between patter


In [20]:
find_quote("new left historians")

Howard Kimeldorf Judith Stepan-Norris Historical Studies of Labor Movements in the United States (1992). Annual Review of Sociology, doc['fpage'], doc['lpage']
Radosh 1966; Aronowitz 1973
ook many diverse forms; the labor movement of the 1930s was far broader and deeper than the strike wave of 1934-1937. Nevertheless, working-class quiescence, even in the turbulent 1930s, remains a central theme in American historiography, with many new left historians portraying the CIO as a conservative bureaucratic monolith, bent on disarming and integrating an otherwise rebellious proletariat ( <CITATION> ). Minus the latent radicalism of the rank and file, this portrayal bears some rather striking simil


In [22]:
find_quote("unreasonable assumptions")

Michael R. Smith What is New in "New Structuralist" Analyses of Earnings? (1990). American Sociological Review, doc['fpage'], doc['lpage']
Bibb and Form 1977; Bibb and Form 1977; Tigges 1987; Tigges 1987; Kalleberg 1988; Kalleberg 1988
 for education. It is true that experience does well in earnings equations (Sorensen 1983, p. 266). But, as we will see, experience has been reinterpreted as ameasure of powerratherthan skill (Kalleberg, Wallace, and Althauser 1981, pp. 658-9). Second, economic analyses rest on all sorts of unreasonable assumptions, the most important of which are labor market homogeneity and perfect competition ( <CITATION> ). If workers competed in a single, perfectly competitive labor market, those with equivalent skills


In [23]:
find_quote("substantial growth in the employment rate of all mothers")

Mark Evan Edwards Uncertainty and the Rise of the Work-Family Dilemma (2001). Journal of Marriage and Family, doc['fpage'], doc['lpage']
Bowen & Finegan 1969; Sweet 1973
 than one third of all mothers with children under age 18 were employed in the late 1940s, and many Americans, anticipating a return to prewar patterns of employed fathers and homemaking mothers, would have predicted that this number would shrink. By the early 1970s, however, social scientists recognized that instead there had been steady, substantial growth in the employment rate of all mothers ( <CITATION> ), and this in the midst of the quintessentially domestic ethos of the 1950s. This observed trend ob


In [24]:
find_quote("creative efforts along these lines")

Stephan Fuchs Jonathan H. Turner "What Makes a Science 'Mature'?: Patterns of Organizational Control in Scientific Production" (1986). Sociological Theory, doc['fpage'], doc['lpage']
Merton 1968; Collins 1975
processes within which these laboratory practices operate. What is needed, therefore, is a more fine-tuned theory of the organizational dynamics of scientific activity that seeks to analyze the structural processes within which the "fact-fabrication," "discursive practices," and "paradigmatic processes" of science operate. There has been, of course, a number of creative efforts along these lines ( <CITATION> ); and so, our proposal is hardly revolutionary. Instead, we propose to extend existing analyses in 


In [35]:
find_quote("the standardized Dean Alienation Scale")

Mary Elizabeth O'Brien Effective Social Environment and Hemodialysis Adaptation: A Panel Analysis (1980). Journal of Health and Social Behavior, doc['fpage'], doc['lpage']
Dean 1961
and responsible before the onset of their chronic illness. The quality of interaction scale measured the relative ease or difficulty of the chronic renal patient's social functioning, such as facility in making new friends or acquaintances and degree of effort to participate in ongoing family relationships. Alienation or anomie was determined through use of the standardized Dean Alienation Scale ( <CITATION> ), which measures alienation in terms of three subcomponents: powerlessness, normlessness, and socia


In [36]:
find_quote("vigorously that social research was not an end in itself ")

Joe R. Feagin Social Justice and Sociology: Agendas for the Twenty-First Century: Presidential Address (2001). American Sociological Review, doc['fpage'], doc['lpage']
Friedrichs 1970
 the first graduate sociology department (at the University of Chicago), listed among the major interests of the journal editors the analysis of "plans for social amelioration" (Small 1895:14). A decade later, Small presented a paper at the American Sociological Society's first meeting in which he argued vigorously that social research was not an end in itself but should serve to improve society ( <CITATION> ). Small was not alone in this commitment. In the first decade or two of U.S. sociology, leading sch


In [40]:
find_quote("This largely theoretical research describes a fractured discipline structured by solidly bound")

James Moody Ryan Light A View from Above: The Evolving Sociological Landscape (2006). The American Sociologist, doc['fpage'], doc['lpage']
Friedrichs 1970; Gouldner 1970; Horowitz 1993; Daipha 2001; Daipha 2001; Griffith et al. 1974; Pool and Kochen 1978; He 1999
ps. Nineteenth century sociologists, such as Durkheim (1966), optimistically evaluated sociology's centrality within the broader scientific landscape. By the end of the twenti eth century, however, numerous prominent scholars predicted the increasing fragmenta tion of the discipline. This largely theoretical research describes a fractured discipline structured by solidly bound, autonomous groups ( <CITATION> ). The overall distribution of peaks has changed with the lowering of the HIV/AIDS work. We see now 


In [41]:
find_quote("No major American or European scholar has yet specialized in research in the sociology of sociology")

James E. Curtis John W. Petras The Sociology of Sociology: Some Lines of Inquiry in the Study of the Discipline (1972). The Sociological Quarterly, doc['fpage'], doc['lpage']
Page 1963; Page 1965; Friedrichs 1970; Gouldner 1970; Reynolds and Reynolds 1970; Tiryakian 1970
ents in research techniques and the rigors of the scientific method rule out significant influences by such factors (Curtis and Petras, 1970a). Some research, however, has begun to evolve in this area, and our purpose in this article is to examine the nature and types of works that have been done. No major American or European scholar has yet specialized in research in the sociology of sociology ( <CITATION> ), and only occasionally are students introduced, in graduate courses, to some of the scattered find


In [42]:
find_quote("researchers describe housework as intrinsically ungratifying")

Chloe E. Bird Catherine E. Ross Houseworkers and Paid Workers: Qualities of the Work and Effects on Personal Control (1993). Journal of Marriage and Family, doc['fpage'], doc['lpage']
Bernard 1972; Gove & Geerken 1977; Gove & Tudor 1973
ntain that housework is associated with fewer validations of work well done than is paid work (Schooler et al., 1984). We expect that the lack of thanks and of recognition decrease perceived control compared to paid work. Work Fulfillment We expect that unpaid household labor is less fulfilling, on average, than paid labor. A number of researchers describe housework as intrinsically ungratifying ( <CITATION> ). Housewives report that housework does not require a great deal of skill or offer much opportunity


In [43]:
find_quote("the respective prevalence rates of mental disorder between the sexes")

Richard L. Meile David Richard Johnson Louis St. Peter Whither Marital Bliss?: Reply to Roberts (1977). Journal of Health and Social Behavior, doc['fpage'], doc['lpage']
Gove and Tudor 1973; Gove and Clancy 1976; Dohrenwend and Dohrenwend 1976
s of mental disorder of men and women, but rather we were interested in the effect of marital roles upon the prevalence of mental symptomatology in women. The only place the male gender is even mentioned is in the introductory paragraph of our

WHITHER MARITAL BLISS 225 paper. Furthermore, the question of the respective prevalence rates of mental disorder between the sexes is still hotly debated ( <CITATION> ) and we fail to see how the eventual outcome of that debate has any relevance to our test of marita


In [44]:
find_quote("The \"human relations\" group ")

Petro Georgiou The Goal Paradigm and Notes Towards a Counter Paradigm (1973). Administrative Science Quarterly, doc['fpage'], doc['lpage']
Roethlisberger and Dickson 1939; Mayo 1945
vior; for example Michels (1962) showed that the stated goals of democratic political parties were substantially divorced from the actual functioning of these parties. Merton (1957) drew attention to the process by which goals were subordinated to means, as organizational members treated rules as ends rather than means for pursuing the substantive organizational goal. The "human relations" group ( <CITATION> ) and many others found that people in organizations were not one-dimensional "officials" or "employ


In [45]:
find_quote("natural system theorists argued that no organization fully")

PEDRO REYES DONALD J. McCARTY Factors Related to the Power of Lower Participants in Educational Organizations: Multiple Perspectives (1990). Sociological Focus, doc['fpage'], doc['lpage']
Gouldner 1959; Mayo 1945; Selznick 1948
l coordination and control of the members' contributions. Thus, a supervisory position is defined as being more powerful than that of an ordinary worker. A hierarchy is struc tured within a formalized process which removes the "personal" element from the power of the office. On the other hand, natural system theorists argued that no organization fully succeeds in controlling all sources of power ( <CITATION> ). Their view

THE POWER OF LOWER PARTICIPANTS IN EDUCATIONAL ORGANIZATIONS 19 suggests that organiz


In [14]:
qstr = "Alexander\s\(2004\)"
q = {"citations":{"$elemMatch":{"$regex":qstr}}}
cits = list(db['cits'].find(q))

#display(Markdown(f"## {' '.join(r)}"))

counts = Counter()
for x in cits:
    counts.update( [y for y in x['citations'] if re.match(qstr, y)] )

print(counts.most_common(5))
cits = list(filter(lambda x:any( re.match(qstr, y) for y in x['citations'] ), cits))
cits = sorted(cits, key=lambda x:-db['docs'].find_one({"_id":x['doc']})['year'])

display(Markdown(
    "\n".join( sorted([print(f"{c['citations']}...{c['contextLeft']} <CITATION> {c['contextRight']}") for c in cits[:15]]) )
))

[('Alexander (2004)', 51)]
['Alexander (2004)']...is "a discursive response to a tear in the social fabric, where the foundations of an established collective identity are shaken by a traumatic occurrence and are in need of renarration and repair." Cultural trauma is not an event per se but an open-ended and contingent process that "allows collectivities to define new forms of moral responsibility and to redirect the course of political action" ( <CITATION> ). Eyerman's (201 1) research on the political assassination of MLK demonstrates how cultural trauma
['Lamont (1992)', 'Zerubavel (1997)', 'Ganz (2000)', 'Alexander (2004)']...ever, did not allow us to test the explanatory mechanism at play. We do so here.
 We begin by considering new work in cultural sociology. The critical juncture in that field is Swidler’s (1986) reconceptualization of culture not as internalized beliefs, norms, and shared values but as “resources that can be put to strategic use” (DiMaggio 1997, p. 265). Swidle

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

# Random Sample

In [18]:
for r, qstr in zip(rows,qs):
    q = {"citations":{"$elemMatch":{"$regex":qstr}}}
    cits = list(db['cits'].find(q))

    display(Markdown(f"## {' '.join(r)}"))

    counts = Counter()
    for x in cits:
        counts.update( [y for y in x['citations'] if re.match(qstr, y)] )

    print(counts.most_common(5))
    cits = list(filter(lambda x:any( re.match(qstr, y) for y in x['citations'] ), cits))

    display(Markdown(
        "\n".join( sorted([format_cit(c) for c in sample(cits, min(30, len(cits)))]) )
    ))

# Ajzen, I, Fishbein, M {Understanding attitudes and predicting social behavior} 1980

[('Ajzen and Fishbein (1980)', 96), ('Ajzen & Fishbein (1980)', 17)]


+ (1984, Social Psychology Quarterly) By far the most prominent and influential of these models has been the Fishbein/Ajzen 
+ (1984, Social Psychology Quarterly) Extensive reviews of this literature are available 
+ (1984, Social Psychology Quarterly) They explicitly argue 
+ (1985, Social Psychology Quarterly) Although Fishbein and Ajzen 
+ (1985, Social Psychology Quarterly) In by far the most dominant paradigm, attitude and social norms are hypothesized to be independent, parallel causes of intentions 
+ (1986, Review of Religious Research) 
+ (1987, Journal of Health and Social Behavior) Thus, among ethnic adolescents, drug use behavior, including the acceptability of cigarette smoking, is influenced in part by these culturally based beliefs, attitudes and values 
+ (1987, Journal of Marriage and Family) A number of researchers have explored predictions of population behaviors such as contraceptive use and childbearing from the perspective of a psychological model proposed by Fishbein and Ajzen 
+ (1988, Journal of Marriage and Family) A person's intention to perform a particular behavior is considered to be the immediate determinant and best predictor of behavior 
+ (1989, Social Psychology Quarterly) Research on this issue relies heavily on the theory of reasoned action, which maintains that attitudes affect behavior indirectly through intentions 
+ (1990, Social Psychology Quarterly) The "theory of reasoned action" represents the most sustained treatment of the intentionbehavior relationship 
+ (1991, Symbolic Interaction) Behavior’s visibility and accessibility may make its definition and measurement seem more obvious than the relatively obscure attitude 
+ (1992, Social Psychology Quarterly) (187) were not the standard measures suggested by Fishbein and Ajzen 
+ (1992, Social Psychology Quarterly) The Theory of Reasoned Action and the Theory of Planned Behavior The theory of reasoned action (henceforth TRA) 
+ (1992, Social Psychology Quarterly) We hypothesized that self-identity would both reflect and influence attitudes, but that it would not show an independent influence on behavioral intentions in regression equations as long as the relevant measures were constructed according to the method suggested by the exponents of the TRA 
+ (1993, Review of Religious Research) Each intervention was labelled by its position in the presented order as "Strategy A," "Strategy B," and "Strategy C." Three 7-point Likert scales were then presented for each of the three interventions in order to assess the three theoretical components of the attitude construct 
+ (1993, Review of Religious Research) Since attitude has been found to be related to treatment outcome 
+ (1994, American Journal of Sociology) Conative attitudes have been operationalized as attitudes that are inferred from past behavior (Rosenberg and Hovland 1960), or from respondents' stated evaluations of performance of various behaviors, as distinguished from their evaluations of the consequences or goals of those behaviors (Ajzen and Fishbein 1980).8 I think that 8 In their influential work, Fishbein and Ajzen 
+ (1996, European Sociological Review) Of course, this is a Pandora's box that cannot be adequately discussed in this paper 
+ (1998, Journal of Marriage and Family) This perspective on fertility determinants rests on a theoretical linkage that runs from background factors shaping reproductive attitudes, which influence preferences, contraceptive use, and ultimately fertility 
+ (2001, Journal for the Scientific Study of Religion) Intentions, social norms, and salient beliefs about consequences were measured on a seven-point unlikely-likely rating scale 
+ (2001, Social Psychology Quarterly) In the social sciences, Fishbein and Ajzen's theories of reasoned action and planned behavior are the most widely used frameworks for relating attitudes to behavior 
+ (2001, Social Psychology Quarterly) Individuals who are inclined toward a behavior that they perceive as nonnormative are likely to be influenced in their intentions by social pressure to avoid that behavior 
+ (2002, Social Psychology Quarterly) The theory of reasoned action and the theory of planned behavior are leading examples of this level of explanation 
+ (2004, European Sociological Review) One such aspect is 'the person's perception of the social pressures put on him' that a certain behaviour should or should not be performed 
+ (2006, Social Indicators Research) The results of this study support the assertions of Ajzen and Fishbein 
+ (2009, Journal of Health and Social Behavior) In studying the development of AAS risk perceptions, it is important to note that percep tions of risk, in general, have consistently been associated with behavioral intentions, as pro posed in models such as the theory of reasoned action 
+ (2011, BMS: Bulletin of Sociological Methodology / Bulletin de Méthodologie Sociologique) For this, two theories that can be integrated in the contextual framework of value-expectancy theories (Esser, 2001: 245-9) came into consideration: Theory of Reasoned Action (TORA)lTheory of Planned Behavior (TOPB) 
+ (2014, Social Forces) The individual-level approach reflects the influential "theory of reasoned action" in psychology 
+ (2014, Social Forces) The individual-level approach reflects the influential "theory of reasoned action" in psychology 

# Alwin, Duane F., Hauser, Robert M. {The Decomposition of Effects in Path Analysis} 1975

[('Alwin and Hauser (1975)', 54), ('Alwin & Hauser (1975)', 4)]


...

[('Woodward (1965)', 135)]


+ (1968, American Sociological Review) By routineness of work we mean the degree to which organizational members have non-uniform work activities 
+ (1970, Administrative Science Quarterly) Yet 
+ (1971, Administrative Science Quarterly) There is apparently a U-shaped curvilinear relationship between technology and certain dimensions of the social structure of organizations, such as the tendency to break down the labor force into small primary groups, and, in general, the tendency toward organic-flexible, participative, informal-as opposed to mechanistichierarchic, formal-systems of management 
+ (1972, Social Forces) The growth of rational bureaucracy with its routinized procedures and its hierarchical decisions based on calculations of efficiency has been attributed to the effectiveness of such a system in producing a limited and stable range of measurable outputs such as standardized material goods 
+ (1972, Social Forces) The growth of rational bureaucracy with its routinized procedures and its hierarchical decisions based on calculations of efficiency has been attributed to the effectiveness of such a system in producing a limited and stable range of measurable outputs such as standardized material goods 
+ (1974, Administrative Science Quarterly) Task variability affects the extent to which work unit activities can be structured in a routinized, systematized, or mechanized way 
+ (1974, Sociology of Education) An apparent exception is the prominence recently given to technology as a major determinant of organizational structure 
+ (1975, Social Forces) In studies of organizations, both hierarchy (administrative ratio) and structured communications (relative size of the clerical staff) seem to be positively related to technological complexity 
+ (1975, Social Forces) In studies of organizations, both hierarchy (administrative ratio) and structured communications (relative size of the clerical staff) seem to be positively related to technological complexity 
+ (1976, American Sociological Review) Blau and The Aston Group provide strong empirical support for the importance of organizational size (Blau, 1972;Pughet al., 1969), but no less persuasive are those arguing for other factors such as environmental uncertainty (Lawrence and Lorsch, 1967; Thompson, 1967) and technology 
+ (1977, Administrative Science Quarterly) Although many typologies exist 
+ (1979, Acta Sociologica) On the one hand there 137//////are research studies which are intimately integrated in the theoretical exposition 
+ (1979, Administrative Science Quarterly) An examination of early attempts to form typologies or taxonomies of organizations reveals that they have had several things in common: First, as noted by McKelvey (1975), most of them were based on only one or two parameters 
+ (1980, Administrative Science Quarterly) The imperative of satisfying the psychological needs of organizational members (Trist and Bamforth, 1951; Argyris, 1952, 1957), and of adopting appropriate managerial styles (McGregor, 1960; Likert, 1967), technology 
+ (1981, Administrative Science Quarterly) Although general organizational theory holds that the structural features of an organization should fit the demands of environment and technology 
+ (1981, Administrative Science Quarterly) Technology 
+ (1981, Administrative Science Quarterly) The analytic technique used in this study has not been widely applied to contingency arguments in the past.1 Previous analyses of contingency ideas have ranged from rich empirical descriptions (Burns and Stalker, 1961) to diagrams and data but no tests of significance 
+ (1982, Journal of Health and Social Behavior) Differences between techno-//////86 JOURNAL OF HEALTH AND SOCIAL BEHAVIOR logical complexity, classified by the production process, e.g., unit, batch, assembly line, or continuous process, were related to differences in structural characteristics of the organization 
+ (1984, Acta Sociologica) Its greatest success is in 170//////continuous-process production 
+ (1984, Administrative Science Quarterly) If the objective was to maximize Supply's ready- 2621ASQ, June 1984//////for-issue assets, then maximum shop output would be desired and could be obtained through batch processing 
+ (1987, Administrative Science Quarterly) Past empirical studies have concluded that both technical complexity and structural complexity are positively related to administrative intensity 
+ (1987, Administrative Science Quarterly) The results suggest that the oft-replicated positive relationship between complexity and administrative intensity 
+ (1987, Sociological Theory) Contingency theory A related but somewhat different view of surveillance draws on the contingency approach to complex organizations 
+ (1989, Social Problems) Engineers are more powerful in these firms because of the high level of uncertainty with which they cope, and because their skills are essential to the organization 
+ (1990, Administrative Science Quarterly) The relationship between technology and structure has been the topic of much writing and research 
+ (1993, Administrative Science Quarterly) Adaptation theories of organizational action, including structural contingency theory 
+ (1995, Annual Review of Sociology) Theories typically placed in the adaptational camp include contingency theory 
+ (1996, American Journal of Sociology) In the late 1960s and early 1970s, sociologists devoted considerable attention to the role of technology in shaping organizations and affecting market outcomes 
+ (2004, Administrative Science Quarterly) Over time, however, to maintain such an "imperative" view of technology (Khandwalla, 1974) in the face of increasingly contradictory findings, researchers from an objectivist tradition have augmented their theories with additional variables on which technological outcomes are contingent, including organizational size, task complexity, and amount of centralization 
+ (2004, Sociological Theory) That is, a more complicated organization reduces the power of the top of the hierarchy (a point in keeping with many studies of organizational control) 

# Yinger, J M {Religion, society and the individual: An introduction to the sociology of religion} 1957

[('Yinger (1957)', 28)]


+ (1968, Sociological Focus) In general, religious change has been defined as a shift from the spontaneous informality of the sectarian group to a formal organization, usually described as a church or denomination, 
+ (1969, American Sociological Review) Members who deviate from orthodox views on any issue are quickly expelled 
+ (1969, Social Problems) Drawing heavily on the work of Emile Durkheim, the way in which religion performs this role is usually summarized as follows 
+ (1970, American Journal of Sociology) At one pole are those who utilize an institutional model 
+ (1970, American Journal of Sociology) CONCLUSIONS Contemporary sociologists of religion have questioned the generalizations ventured by earlier theorists regarding the organizational changes which sectarian movements typically undergo in the course of their institutionalization over time 
+ (1970, American Journal of Sociology) Some writers have questioned the alleged inevitability of such declining rigor and have pointed to the retention of exacting standards of admission as an important feature of the "institutionalized sect" 
+ (1970, Journal for the Scientific Study of Religion) It tests selected hypotheses derived from descriptions of the church-sect typology 
+ (1971, American Journal of Sociology) There is also a large literature which suggests that conditions of social deprivation 
+ (1971, Review of Religious Research) A number of explanations for the commonly observed sex difference have been proposed 
+ (1971, Review of Religious Research) Sex.-Despite a few studies in which//////50 REVIEW OF RELIGIOUS RESEARCH the sexes were found to differ little in religiosity (Vernon, 1956; Sklare, 1958; Hadden, 1963),21 investigators have commonly found that females are more religious than males 
+ (1974, Journal for the Scientific Study of Religion) It is now understood that sects can develop along a number of lines 
+ (1974, Journal for the Scientific Study of Religion) This approach makes it possible to identify the people who are most likely to experience particular problems and find the movement's solution appealing because of their structural location, prior experiences and problem-solving perspectives 
+ (1974, The Sociological Quarterly) Several investigators of the relation of religion to socioeconomic status have found considerable variation between Protestant denominations (Warren, 1970; Featherman, 1971), and sociologists of religion have pointed to distinctive social behaviors and religious practices within//////Religion and Political Behavior 53 Protestantism 
+ (1975, American Sociological Review) A body of literature emerged which was interpreted to support the contention that religious involvement functioned to assuage the problems associated with isolation and loneliness 
+ (1975, Review of Religious Research) For an almost equal length of time, moreover, the framework has been subjected to critical revision with a number of outstanding proponents making various modifications on the original dichotomy, 
+ (1984, Review of Religious Research) Religion, according to this perspective, has lost its "presence" as a social force and has become less visible, less public, and less important in the lives of people 
+ (1987, Social Forces) Beyond its use as a general orienting proposition, secularization has been used in many ways ranging from the "decline" or "loss" of religion 
+ (1987, Social Forces) Beyond its use as a general orienting proposition, secularization has been used in many ways ranging from the "decline" or "loss" of religion 
+ (1987, Sociological Bulletin) Two social scientists, a foreigner 
+ (1994, Journal for the Scientific Study of Religion) Historically, religion has been seen by theoreticians as a powerful factor promoting good health among both individuals 
+ (1998, Journal for the Scientific Study of Religion) INTRODUCTION Church-sect theory, when applied, as it typically has been, to sects in single societies where they began schismatically, has proved to be full of insights concerning the development of sects/new religous movements 
+ (2001, Review of Religious Research) But, as the proponents of this definition frankly admit, "Religion, of course, is not alone in attempting to deal with the ultimate problems of human life" 
+ (2001, Review of Religious Research) In this tradition, religion is any system of beliefs and practices involving answers to questions about the meaning and purpose of life 
+ (2001, Review of Religious Research) Many others have followed Durkheim ([1912] 1995) in distinguishing religion from magic on the basis their goals, identifying religion as being mainly concerned with general, long-range, goals and magic as concerned with immedi- 102//////ate and concrete goals 
+ (2009, Journal for the Scientific Study of Religion) The debate about the decline-ofreligion thesis is about whether, where, and why we are witnessing a decline of institutionalized, mainline church religion on the one hand (e.g., Houtman and Mascini 2002; Noms and Inglehart 2004) and religious innovation, revitalization, and cult formation, typically occurring outside the mainline churches, on the other 

# MOST RECENT

In [39]:
for r, qstr in zip(rows,qs):
    q = {"citations":{"$elemMatch":{"$regex":qstr}}}
    cits = list(db['cits'].find(q))

    display(Markdown(f"## {' '.join(r)}"))

    counts = Counter()
    for x in cits:
        counts.update( [y for y in x['citations'] if re.match(qstr, y)] )

    print(counts.most_common(5))
    cits = list(filter(lambda x:any( re.match(qstr, y) for y in x['citations'] ), cits))
    cits = sorted(cits, key=lambda x:-db['docs'].find_one({"_id":x['doc']})['year'])

    display(Markdown(
        "\n".join( sorted([format_cit(c) for c in cits[:10]]) )
    ))

## Ajzen, I, Fishbein, M {Understanding attitudes and predicting social behavior} 1980

[('Ajzen and Fishbein (1980)', 96), ('Ajzen & Fishbein (1980)', 17)]


+ (2009, Acta Sociologica) Perhaps the two dominant theoretical perspectives on the influence of attitudes or values on demographic tran sitions are (a) Fishbein and Ajzen's theory of Reasoned Action (1975) and Planned Behaviour 
+ (2009, Sociology) In contrast, psychological theories of intention, and, in particular, the Theory of Planned Behaviour 
+ (2010, Social Psychology Quarterly) The present article advances work on the attitude-behavior linkage in behavioral cas cades via a joint consideration of two theories: the approach to the attitude-behavior linkage advanced by the theory of reasoned-planned action 
+ (2011, BMS: Bulletin of Sociological Methodology / Bulletin de Méthodologie Sociologique) For this, two theories that can be integrated in the contextual framework of value-expectancy theories (Esser, 2001: 245-9) came into consideration: Theory of Reasoned Action (TORA)lTheory of Planned Behavior (TOPB) 
+ (2014, Social Forces) The individual-level approach reflects the influential "theory of reasoned action" in psychology 
+ (2014, Social Forces) The individual-level approach reflects the influential "theory of reasoned action" in psychology 
+ (2015, American Journal of Sociology) We use the theory of reasoned action to conceptualize how endorsement of DI increases the likelihood that an individual chooses his or her own, potentially different caste, spouse 
+ (2015, American Journal of Sociology) We use the theory of reasoned action to conceptualize how endorsement of DI increases the likelihood that an individual chooses his or her own, potentially different caste, spouse 
+ (2015, Social Forces) Cultural and political circumstances or norms can increase the legitimacy of employing a specific type of argumentation 
+ (2015, Social Forces) Cultural and political circumstances or norms can increase the legitimacy of employing a specific type of argumentation 

## Alwin, Duane F., Hauser, Robert M. {The Decomposition of Effects in Path Analysis} 1975

[('Alwin and Hauser (1975)', 54), ('Alwin & Hauser (1975)', 4)]


...

[('Woodward (1965)', 135)]


+ (1997, Sociology) In contrast to previous studies, which have sought to explain supervisory roles in terms of their link with structural factors such as technology, organisational size and formalisation 
+ (1997, Sociology) Previous studies have established the importance of various structural factors in shaping supervisory systems, for example the size of the organisation, process technology, payment systems, and so on 
+ (1999, Annual Review of Sociology) Some underlying propositions that apply to an industrial technology determinist view are: (a) Technology is the solution to the problems organizations face, particularly with respect to competitiveness; (b) organizations must adapt to accommodate the technology, after it is installed; (c) technology type determines the best organizational design 
+ (1999, Sociological Perspectives) HYPOTHESES: ORGANIZATIONAL CHARACTERISTICS THAT DROVE ADOPTION Size and Complexity Several scholars have explored the association between organizational complexity and technology 
+ (2002, American Sociological Review) This seems an especially reasonable assumption in industries like chemical production, which depend on continuous process systems 
+ (2004, Administrative Science Quarterly) Over time, however, to maintain such an "imperative" view of technology (Khandwalla, 1974) in the face of increasingly contradictory findings, researchers from an objectivist tradition have augmented their theories with additional variables on which technological outcomes are contingent, including organizational size, task complexity, and amount of centralization 
+ (2004, Sociological Theory) That is, a more complicated organization reduces the power of the top of the hierarchy (a point in keeping with many studies of organizational control) 
+ (2011, Sociology of Education) 2007), few program designs that aim at technol ogy implementation have seriously considered the complexity of teaching 
+ (2011, Sociology of Education) As a result, teaching is complex because teachers must both adapt practices to local contexts and coordinate with each other as they do so 
+ (2013, The American Sociologist) Likewise, in Contingency theory the object of analysis is primarily profitoriented organizations 

## Yinger, J M {Religion, society and the individual: An introduction to the sociology of religion} 1957

[('Yinger (1957)', 28)]


+ (1984, Review of Religious Research) Religion, according to this perspective, has lost its "presence" as a social force and has become less visible, less public, and less important in the lives of people 
+ (1987, Social Forces) Beyond its use as a general orienting proposition, secularization has been used in many ways ranging from the "decline" or "loss" of religion 
+ (1987, Social Forces) Beyond its use as a general orienting proposition, secularization has been used in many ways ranging from the "decline" or "loss" of religion 
+ (1987, Sociological Bulletin) Two social scientists, a foreigner 
+ (1994, Journal for the Scientific Study of Religion) Historically, religion has been seen by theoreticians as a powerful factor promoting good health among both individuals 
+ (1998, Journal for the Scientific Study of Religion) INTRODUCTION Church-sect theory, when applied, as it typically has been, to sects in single societies where they began schismatically, has proved to be full of insights concerning the development of sects/new religous movements 
+ (2001, Review of Religious Research) But, as the proponents of this definition frankly admit, "Religion, of course, is not alone in attempting to deal with the ultimate problems of human life" 
+ (2001, Review of Religious Research) In this tradition, religion is any system of beliefs and practices involving answers to questions about the meaning and purpose of life 
+ (2001, Review of Religious Research) Many others have followed Durkheim ([1912] 1995) in distinguishing religion from magic on the basis their goals, identifying religion as being mainly concerned with general, long-range, goals and magic as concerned with immedi- 102//////ate and concrete goals 
+ (2009, Journal for the Scientific Study of Religion) The debate about the decline-ofreligion thesis is about whether, where, and why we are witnessing a decline of institutionalized, mainline church religion on the one hand (e.g., Houtman and Mascini 2002; Noms and Inglehart 2004) and religious innovation, revitalization, and cult formation, typically occurring outside the mainline churches, on the other 